# Installation

In [1]:
# !pip install tabula-py[jpype]

# Imports

In [2]:
import os
import shutil
import numpy as np
import pandas as pd
from tabula import read_pdf

# Constants

In [3]:
RAW_FILE_DIRECTORY = r"..\data\pdf"

PROCESSED_FILE_DIRECTORY = r"..\data\excel"

ERROR_FILE_DIRECTORY = r"..\data\errorpdf"

data = {}

# Data Extraction

## Get Files

In [4]:
files = os.listdir(RAW_FILE_DIRECTORY)

files = [f for f in files if os.path.isfile(RAW_FILE_DIRECTORY + '/' + f)]

In [5]:
print(files)

['IR-M-C-19343.pdf', 'IR-M-C-32855.pdf', 'IR-M-C-32887.pdf', 'IR-M-C-34129.pdf', 'IR-M-C-36557.pdf', 'IR-M-C-37013.pdf', 'IR-M-C-56616.pdf', 'IR-M-I-1044.pdf', 'IR-M-I-1064.pdf', 'IR-M-I-1066.pdf', 'IR-M-I-1074.pdf', 'IR-M-I-1075.pdf', 'IR-M-I-1124.pdf', 'IR-M-I-1132.pdf', 'IR-M-I-1170.pdf', 'IR-M-I-1238.pdf', 'IR-M-I-1408.pdf', 'IR-M-I-1441.pdf', 'IR-M-I-1480.pdf', 'IR-M-N-10.pdf', 'IR-M-N-16.pdf', 'IR-M-N-4.pdf', 'IR-M-S-132.pdf', 'IR-M-S-14553.pdf', 'IR-M-S-14565.pdf', 'IR-M-S-14875.pdf', 'IR-M-S-156.pdf', 'IR-M-S-15911.pdf', 'IR-M-S-15962.pdf', 'IR-M-S-164.pdf', 'IR-M-S-16881.pdf', 'IR-M-S-17295.pdf', 'IR-M-S-309.pdf', 'IR-M-S-3283.pdf', 'IR-M-S-354.pdf', 'IR-M-S-394.pdf', 'IR-M-S-419.pdf', 'IR-M-S-6.pdf', 'IR-M-S-87.pdf', 'IR-M-S-8868.pdf', 'IR-M-S-8890.pdf', 'IR-M-S-8895.pdf', 'IR-M-S-8903.pdf', 'IR-M-S-8904.pdf', 'IR-M-S-8909.pdf', 'IR-M-S-8918.pdf', 'IR-M-S-8931.pdf', 'IR-M-S-8937.pdf', 'IR-M-S-8942.pdf', 'IR-M-S-8948.pdf', 'IR-M-S-8959.pdf', 'IR-M-S-8967.pdf', 'IR-M-S-8972.pdf

In [6]:
len(files)

100

## Read the file and export the data to Excel

In [25]:
table_first_columns = ['Academic Year'
                       ,'(All programs\rof all years)'
                       ,'Academic Year'
                       ,'Ph.D (Student pursuing doctoral program till 2021-22)'
                       ,'Financial Year'
                       ,'Financial Year'
                       ,'Financial Year'
                       ,'Financial Year'
                       ,'Financial Year'
                       ,'1. Do your institution buildings have Lifts/Ramps?'
                       ,'Number of faculty members entered'
                      ]

In [54]:
nirf_df = read_pdf(RAW_FILE_DIRECTORY + "/" + 'IR-M-C-32855.pdf', pages='all', guess = False)

In [55]:
len(nirf_df)

11

In [51]:
nirf_df[0].columns[0], nirf_df[1].columns[0], nirf_df[2].columns[0], nirf_df[3].columns[0], nirf_df[4].columns[0]\
, nirf_df[5].columns[0], nirf_df[6].columns[0], nirf_df[7].columns[0], nirf_df[8].columns[0], nirf_df[9].columns[0] \
, nirf_df[10].columns[0]

('Academic Year',
 '(All programs\rof all years)',
 'Academic Year',
 'Ph.D (Student pursuing doctoral program till 2021-22)',
 'Financial Year',
 'Financial Year',
 'Financial Year',
 'Financial Year',
 'Financial Year',
 '1. Do your institution buildings have Lifts/Ramps?',
 'Number of faculty members entered')

In [53]:
for file in files:   
    if file.endswith('.pdf'):
        try:
            nirf_df = read_pdf(RAW_FILE_DIRECTORY + "/" + file, pages='all', guess = False)

            #print(f"Extracting the data from the file {file}. The file has {len(nirf_df)} tables.")

            nirf_data = []
            table_index = 0

            for df in nirf_df:           
                if df.columns[0] == table_first_columns[table_index]:
                    if table_index != 9 and table_index != 10:
                        nirf_data.append(df)                                   
                    else:
                        data_list = [df.columns.values.tolist()]

                        for value in df.values.tolist():
                            data_list.append(value)

                        data_df = pd.DataFrame(data_list, columns=['Question', 'Answer'])

                        nirf_data.append(data_df)

                    table_index += 1
                else:
                    table_index -= 1

                    previous_df = nirf_data[table_index]

                    columns = previous_df.columns.values.tolist()

                    data_list = previous_df.values.tolist()

                    data_list.append(df.columns.values.tolist())

                    for value in df.values.tolist():
                        data_list.append(value)                

                    nirf_data[table_index] = pd.DataFrame(data_list, columns=columns)

                    table_index += 1

            #data[(file.split('-')[3]).split('.')[0]] = nirf_data       

            file_name = file.replace(".pdf", '.xlsx')

            with pd.ExcelWriter(PROCESSED_FILE_DIRECTORY + "/" + file_name) as writer:
                for index in range(len(nirf_data)):
                    nirf_data[index]['Institution'] = (file.split('.')[0])
                    nirf_data[index].to_excel(writer, sheet_name=str(index), index=False)
        except Exception as ex:
            print(f"Error while processing {file}.\n{ex}")
            #shutil.move(RAW_FILE_DIRECTORY + "/" + file, ERROR_FILE_DIRECTORY + "/" + file)
            continue

print("Completed the data extraction.")

Error while processing IR-M-C-32855.pdf.
list index out of range
Error while processing IR-M-C-37013.pdf.
list index out of range
Error while processing IR-M-I-1066.pdf.
4 columns passed, passed data had 6 columns
Error while processing IR-M-I-1074.pdf.
4 columns passed, passed data had 6 columns
Error while processing IR-M-I-1075.pdf.
list index out of range
Error while processing IR-M-I-1170.pdf.
4 columns passed, passed data had 6 columns
Error while processing IR-M-I-1238.pdf.
list index out of range
Error while processing IR-M-I-1408.pdf.
list index out of range
Error while processing IR-M-I-1441.pdf.
list index out of range
Error while processing IR-M-I-1480.pdf.
list index out of range
Error while processing IR-M-N-4.pdf.
list index out of range
Error while processing IR-M-S-132.pdf.
list index out of range
Error while processing IR-M-S-14553.pdf.
list index out of range
Error while processing IR-M-S-14875.pdf.
list index out of range
Error while processing IR-M-S-156.pdf.
4 col